In [9]:
import pandas as pd

In [10]:
# Cargar los archivos CSV
df1 = pd.read_csv('/mnt/c6c29b92-c396-4913-b7c7-b2456280d17a/Software y Programación/Ciencia de Datos/Datathon/ProdEnPromo-UM.csv')
df2 = pd.read_csv('/mnt/c6c29b92-c396-4913-b7c7-b2456280d17a/Software y Programación/Ciencia de Datos/Datathon/Producto-UM.csv')
df3 = pd.read_csv('/mnt/c6c29b92-c396-4913-b7c7-b2456280d17a/Software y Programación/Ciencia de Datos/Datathon/VentasPR.csv')

# Ahora df1, df2, y df3 son DataFrames de pandas que contienen los datos de cada archivo CSV respectivamente.

In [11]:
df1

,PRDCODEXT,PROMDEFFECDES,PROMDEFFECHAS
0,560226,13/02/2023,26/02/2023
1,560227,13/02/2023,26/02/2023
2,561875,13/02/2023,26/02/2023
3,561895,13/02/2023,26/02/2023
4,561899,13/02/2023,26/02/2023
...,...,...,...
1689,607064,06/02/2024,25/02/2024
1690,607065,06/02/2024,25/02/2024
1691,607066,06/02/2024,25/02/2024
1692,560226,19/03/2024,07/04/2024


In [12]:
df2

,Cod,Status Hoy,Prov,Cat,Tipo Prov
0,561699,5,68952,216,1
1,561634,5,68952,216,1
2,561694,5,68952,216,1
3,561826,5,68952,216,1
4,561891,5,68952,216,1
...,...,...,...,...,...
2079,562769,5,66468,216,1
2080,562771,5,66468,216,1
2081,562772,5,66468,216,1
2082,562774,5,66468,216,1


In [13]:
df3

,LOCCOD,MOVFEC,PRDCODEXT,ACPRCANTVEND,ACPRIMPVEN
0,5201,2/5/2021,600790,7,1393
1,5201,2/5/2021,600793,2,390
2,5201,2/5/2021,600794,7,2023
3,5201,2/5/2021,605089,1,399
4,5201,2/5/2021,605097,1,565
...,...,...,...,...,...
145271,5201,30/4/2024,600917,4,2320
145272,5201,30/4/2024,600918,1,580
145273,5201,30/4/2024,600919,1,580
145274,5201,30/4/2024,600922,1,379


In [14]:
# Renombrar la columna 'Cod' en df2 a 'PRDCODEXT'
df2.rename(columns={'Cod': 'PRDCODEXT'}, inplace=True)

# Agregar sufijos a las columnas de cada dataframe para identificar su origen
df1 = df1.add_suffix('_df1')
df2 = df2.add_suffix('_df2')
df3 = df3.add_suffix('_df3')

# Corregir los nombres de las columnas de PRDCODEXT para que coincidan en todos los dataframes
df1.rename(columns={'PRDCODEXT_df1': 'PRDCODEXT'}, inplace=True)
df2.rename(columns={'PRDCODEXT_df2': 'PRDCODEXT'}, inplace=True)
df3.rename(columns={'PRDCODEXT_df3': 'PRDCODEXT'}, inplace=True)


In [19]:
# Realizar un full outer join
df_merged = df1.merge(df2, on='PRDCODEXT', how='outer')
df_merged = df_merged.merge(df3, on='PRDCODEXT', how='outer')

In [20]:
df_merged

,PRDCODEXT,PROMDEFFECDES_df1,PROMDEFFECHAS_df1,Status Hoy_df2,Prov_df2,Cat _df2,Tipo Prov_df2,LOCCOD_df3,MOVFEC_df3,ACPRCANTVEND_df3,ACPRIMPVEN_df3
0,560226,13/02/2023,26/02/2023,1.0,10214.0,216.0,1.0,5201.0,17/8/2021,2.0,978
1,560226,13/02/2023,26/02/2023,1.0,10214.0,216.0,1.0,5201.0,23/8/2021,0.0,489
2,560226,13/02/2023,26/02/2023,1.0,10214.0,216.0,1.0,5201.0,24/8/2021,1.0,489
3,560226,13/02/2023,26/02/2023,1.0,10214.0,216.0,1.0,5201.0,27/8/2021,4.0,1956
4,560226,13/02/2023,26/02/2023,1.0,10214.0,216.0,1.0,5201.0,28/8/2021,1.0,489
...,...,...,...,...,...,...,...,...,...,...,...
344743,600684,NaN,NaN,NaN,NaN,NaN,NaN,5201.0,4/11/2023,1.0,219
344744,600684,NaN,NaN,NaN,NaN,NaN,NaN,5201.0,15/11/2023,1.0,219
344745,600684,NaN,NaN,NaN,NaN,NaN,NaN,5201.0,18/11/2023,1.0,219
344746,600684,NaN,NaN,NaN,NaN,NaN,NaN,5201.0,2/12/2023,1.0,219


In [24]:
# Exportar el dataframe combinado a un archivo CSV
df_merged.to_csv('dataframe_combinado.csv', index=False)